# KW_MMDS - Colab 1
## Wordcount in Spark

### Setup

Let's setup Spark on your Colab environment.  Run the cell below!

In [1]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     |████████████████████████████████| 204.2MB 66kB/s 
     |████████████████████████████████| 204kB 46.3MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612243 sha256=30ddc4063a0a54257db3aef01538477824a0eede3672c3c9cc5a8f0b3f9ffa00
  Stored in directory: /root/.cache/pip/wheels/5e/bd/07/031766ca628adec8435bb40f0bd83bb676ce65ff4007f8e73f
Successfully built pyspark
The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 2 newly installed, 0 to remove and 11 not upgraded.
Need to get 35.8 MB of archives.
After this operation, 140 MB of additional disk space will be used.
Selecting previously unselected package openjdk-8-jre-headless:amd64.
(Reading

Now we authenticate a Google Drive client to download the file we will be processing in our Spark job.

**Make sure to follow the interactive instructions.**

In [2]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [3]:
id='1SE6k_0YukzGd5wK-E4i6mG83nydlfvSa'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('pg100.txt')

If you executed the cells above, you should be able to see the file *pg100.txt* under the "Files" tab on the left panel.

### Your task

If you run successfully the setup stage, you are ready to work on the *pg100.txt* file which contains a copy of the complete works of Shakespeare.

1. Write a Spark application which outputs the number of words that start with each letter. This means that for every letter we want to count the total number of (non-unique) words that start with a specific letter. In your implementation **ignore the letter case**, i.e., consider all words as lower case. Also, you can ignore all the words **starting** with a non-alphabetic character.

In [4]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
import pandas as pd

# create the Spark Session
spark = SparkSession.builder.getOrCreate()

# create the Spark Context
sc = spark.sparkContext

In [7]:
# YOUR CODE HERE, USE THIS CELL ONLY (DO NOT USE ADDITIONAL CELLS.)
# 한 번에 몰아서 하기
RDDs = sc.textFile('pg100.txt').map(lambda line: line.lower())\
                              .flatMap(lambda line: line.split(" "))\
                              .filter(lambda word: len(word) > 0)\
                              .filter(lambda word: word[0].isalpha())\
                              .map(lambda word: (word[0], 1))\
                              .reduceByKey(lambda a, b: a + b)
RDDs.take(26)

[('p', 27759),
 ('g', 20782),
 ('c', 34567),
 ('s', 65705),
 ('b', 45455),
 ('i', 62167),
 ('r', 14265),
 ('y', 25855),
 ('l', 29569),
 ('d', 29713),
 ('j', 3339),
 ('h', 60563),
 ('t', 123602),
 ('e', 18697),
 ('o', 43494),
 ('w', 59597),
 ('f', 36814),
 ('u', 9170),
 ('a', 84836),
 ('n', 26759),
 ('m', 55676),
 ('v', 5728),
 ('k', 9418),
 ('q', 2377),
 ('z', 71),
 ('x', 14)]

2. Write a Spark application which outputs the 10 most frequently used words contained in 'the Pride and Prejudice' by Jane Austen. In your implementation **ignore the letter case**, i.e., consider all words as **upper** case.

In [8]:
# 오만과 편견 다운로드: 1342-0.txt
pride_id='11HHiWgRiXHuNehW8AXrHarKJfGX1HHnB'
pride_downloaded = drive.CreateFile({'id': pride_id})
pride_downloaded.GetContentFile('1342-0.txt')

In [9]:
# YOUR CODE HERE, USE THIS CELL ONLY (DO NOT USE ADDITIONAL CELLS.)
# 한 번에 몰아서 하기
RDDs = sc.textFile('1342-0.txt').map(lambda line: line.upper())\
                              .flatMap(lambda line: line.split(" "))\
                              .filter(lambda word: len(word) > 0)\
                              .filter(lambda word: word[0].isalpha())\
                              .map(lambda word: (word, 1))\
                              .reduceByKey(lambda a, b: a + b)\
                              .map(lambda tuple: (tuple[1], tuple[0]))\
                              .sortByKey(ascending=False)
RDDs.take(10)

[(4480, 'THE'),
 (4169, 'TO'),
 (3681, 'OF'),
 (3400, 'AND'),
 (1981, 'A'),
 (1939, 'HER'),
 (1890, 'IN'),
 (1797, 'WAS'),
 (1725, 'I'),
 (1607, 'SHE')]